In [42]:
import torch
import torch.nn.functional as F
import numpy as np
import math

def relu(x):
    x[x < 0] = 0
    
    return x

def loss_func(x, y):
    f = -torch.mean(torch.log(x[torch.arange(list(x.size())[0]), y]))
    print('loss_func = ', f.item())
    return f

class Net():
    def __init__(self, x, y):
        self.input = x
        self.weights1 = torch.rand(54,13) 
        self.weights2 = torch.rand(13,7)
        self.b1 = torch.rand(1,13) 
        self.b2 = torch.rand(1,7)
        self.y = y
        self.output = torch.zeros(y.shape)
        self.backprop()
        
    def feedforward(self, weights1, weights2, b1, b2):
        self.layer1 = relu(torch.add(torch.mm(self.input, weights1), b1))
        self.output = torch.softmax(torch.add(relu(torch.mm(self.layer1, weights2)), b2), dim = 1)
        
        return self.output
    
    def backprop(self):
        self.weights1 = torch.tensor(self.weights1, requires_grad=True)
        self.weights2 = torch.tensor(self.weights2, requires_grad=True)
        self.b1 = torch.tensor(self.b1, requires_grad=True) 
        self.b2 = torch.tensor(self.b2, requires_grad=True)
        lr = 0.02
        
        for iteration in range(1000):
            with torch.no_grad():
                if self.weights1.grad is not None:
                    self.weights1.grad.zero_()
            with torch.no_grad():
                if self.weights2.grad is not None:
                    self.weights2.grad.zero_()
            with torch.no_grad():
                if self.b1.grad is not None:
                    self.b1.grad.zero_()
            with torch.no_grad():
                if self.b2.grad is not None:
                    self.b2.grad.zero_()
                    
            f = loss_func(self.feedforward(self.weights1, self.weights2, self.b1, self.b2), self.y)
            f.backward()  
            
            with torch.no_grad():
                 self.weights1 -= lr * self.weights1.grad
            with torch.no_grad():        
                 self.weights2 -= lr * self.weights2.grad
            with torch.no_grad():
                 self.b1 -= lr * self.b1.grad
            with torch.no_grad():        
                 self.b2 -= lr * self.b2.grad
        
    
    def guess(self, x, y):
        self.layer1 = relu(torch.add(torch.mm(x, self.weights1), self.b1))
        self.output = torch.softmax(torch.add(relu(torch.mm(self.layer1, self.weights2)), self.b2), dim = 1)
        self.result = torch.argmax(self.output, dim = 1)
        
        print(self.result)
    
    def accuracy(self, y):
        count = list(self.result.size())[0]
        
        for num in range(0, list(self.result.size())[0]):
            if self.result[num] != y[num]:
                count -= 1
                
        print('accuracy = ' , count / list(self.result.size())[0] , '%')

In [43]:
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader
import pandas as pd

covertype = fetch_openml(data_id=180)
cover_df = pd.DataFrame(data=covertype.data, columns=covertype.feature_names)
label_encoder = LabelEncoder().fit(covertype.target)
cover_target = label_encoder.transform(covertype.target)
df_train, df_test, y_train, y_test = train_test_split(cover_df, cover_target, test_size=0.15, stratify=cover_target)

to_normalize = [(i, col) for i, col in enumerate(cover_df.columns)
                        if not col.startswith('wilderness_area') and not col.startswith('soil_type')]
idx_to_normalize = [i for i,col in to_normalize] #номера столбцов
columns_to_normalize = [col for i, col in to_normalize] #названия

tensor_train = torch.from_numpy(df_train.values).type(torch.FloatTensor)
tensor_test = torch.from_numpy(df_test.values).type(torch.FloatTensor)
train_mean = torch.mean(tensor_train[:,idx_to_normalize], dim=0)
train_std = torch.std(tensor_train[:,idx_to_normalize], dim=0)

tensor_train[:,idx_to_normalize] -= train_mean
tensor_train[:,idx_to_normalize] /= train_std
tensor_test[:,idx_to_normalize] -= train_mean
tensor_test[:,idx_to_normalize] /= train_std

y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)
y_test_tensor = torch.from_numpy(y_test).type(torch.LongTensor)
train_ds = TensorDataset(tensor_train, y_train_tensor)
test_ds = TensorDataset(tensor_test, y_test_tensor)
train_loader = DataLoader(train_ds,batch_size=512, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=512, shuffle=True)

In [44]:
net = Net(tensor_train, y_train_tensor)
    

C:\Users\GudWin\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\GudWin\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\GudWin\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\GudWin\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach(

loss_func =  2.8554515838623047
loss_func =  2.3385674953460693
loss_func =  2.014789342880249
loss_func =  1.8501337766647339
loss_func =  1.7645639181137085
loss_func =  1.7134584188461304
loss_func =  1.6793618202209473
loss_func =  1.6546810865402222
loss_func =  1.635698914527893
loss_func =  1.6204159259796143
loss_func =  1.6076664924621582
loss_func =  1.5967282056808472
loss_func =  1.5871295928955078
loss_func =  1.578551173210144
loss_func =  1.5707682371139526
loss_func =  1.563619613647461
loss_func =  1.5569854974746704
loss_func =  1.5507756471633911
loss_func =  1.5449222326278687
loss_func =  1.5393718481063843
loss_func =  1.5340821743011475
loss_func =  1.5290193557739258
loss_func =  1.5241554975509644
loss_func =  1.5194693803787231
loss_func =  1.5149422883987427
loss_func =  1.5105584859848022
loss_func =  1.5063062906265259
loss_func =  1.502173662185669
loss_func =  1.4981507062911987
loss_func =  1.4942296743392944
loss_func =  1.490403413772583
loss_func =  1

loss_func =  1.1641302108764648
loss_func =  1.1634657382965088
loss_func =  1.1628042459487915
loss_func =  1.1621458530426025
loss_func =  1.1614899635314941
loss_func =  1.1608372926712036
loss_func =  1.160187005996704
loss_func =  1.1595396995544434
loss_func =  1.158895492553711
loss_func =  1.1582539081573486
loss_func =  1.157615303993225
loss_func =  1.1569795608520508
loss_func =  1.1563467979431152
loss_func =  1.1557164192199707
loss_func =  1.1550889015197754
loss_func =  1.1544640064239502
loss_func =  1.1538418531417847
loss_func =  1.1532220840454102
loss_func =  1.1526050567626953
loss_func =  1.1519911289215088
loss_func =  1.1513795852661133
loss_func =  1.1507707834243774
loss_func =  1.1501648426055908
loss_func =  1.1495612859725952
loss_func =  1.1489602327346802
loss_func =  1.1483615636825562
loss_func =  1.1477653980255127
loss_func =  1.1471718549728394
loss_func =  1.1465805768966675
loss_func =  1.1459918022155762
loss_func =  1.1454054117202759
loss_func =

loss_func =  1.0551249980926514
loss_func =  1.0548615455627441
loss_func =  1.0545986890792847
loss_func =  1.0543367862701416
loss_func =  1.0540755987167358
loss_func =  1.0538150072097778
loss_func =  1.0535554885864258
loss_func =  1.053296446800232
loss_func =  1.0530383586883545
loss_func =  1.0527808666229248
loss_func =  1.0525239706039429
loss_func =  1.0522680282592773
loss_func =  1.0520126819610596
loss_func =  1.0517582893371582
loss_func =  1.0515047311782837
loss_func =  1.0512515306472778
loss_func =  1.0509992837905884
loss_func =  1.0507477521896362
loss_func =  1.0504966974258423
loss_func =  1.0502465963363647
loss_func =  1.049997329711914
loss_func =  1.0497485399246216
loss_func =  1.049500584602356
loss_func =  1.049253225326538
loss_func =  1.0490063428878784
loss_func =  1.0487604141235352
loss_func =  1.0485150814056396
loss_func =  1.0482702255249023
loss_func =  1.0480259656906128
loss_func =  1.04778254032135
loss_func =  1.0475397109985352
loss_func =  1

loss_func =  1.0050652027130127
loss_func =  1.0049223899841309
loss_func =  1.0047796964645386
loss_func =  1.0046371221542358
loss_func =  1.0044951438903809
loss_func =  1.004353404045105
loss_func =  1.0042119026184082
loss_func =  1.00407075881958
loss_func =  1.0039299726486206
loss_func =  1.0037891864776611
loss_func =  1.0036489963531494
loss_func =  1.0035088062286377
loss_func =  1.0033692121505737
loss_func =  1.0032296180725098
loss_func =  1.0030906200408936
loss_func =  1.002951741218567
loss_func =  1.0028132200241089
loss_func =  1.0026750564575195
loss_func =  1.0025368928909302
loss_func =  1.0023993253707886
loss_func =  1.002261757850647
loss_func =  1.0021247863769531
loss_func =  1.0019879341125488
loss_func =  1.0018515586853027
loss_func =  1.0017151832580566
loss_func =  1.0015792846679688
loss_func =  1.0014433860778809
loss_func =  1.0013078451156616
loss_func =  1.0011725425720215
loss_func =  1.001037359237671
loss_func =  1.0009024143218994
loss_func =  1

In [48]:
print(y_test_tensor)
net.guess(tensor_test, y_test_tensor)
net.accuracy(y_test_tensor)


tensor([5, 4, 3,  ..., 4, 6, 4])
tensor([4, 4, 4,  ..., 6, 6, 4])
accuracy =  0.653843831149224 %
